In [2]:
# Author: Ruojia Sun, parts of the code are written by Prof. Ami Gates

# import PyPDF2
import pandas as pd
import os
import requests  ## for getting data from a server GET
import re   ## for regular expressions
import numpy as np

In [3]:
# assign directory
directory = 'events-by-state'
file_list = []
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        file_list.append(f)

In [ ]:
# reader = PyPDF2.PdfReader('us-climate-disasters.pdf')
cols = ["Name","Disaster","Begin Date","End Date","Total CPI-Adjusted Cost (Millions of Dollars)","Deaths","State"]
df = pd.DataFrame(columns = cols)

for file in file_list:
    df_new = pd.read_csv(file,skiprows=1)
    state = file.split('-')[3]
    df_new['State'] = [state]*len(df_new.index)
    for i_row in range(len(df_new.index)):
        search = df[df['Name']==df_new.loc[i_row,'Name']]
        if not search.empty:
            df.loc[search.index[0],'State']+= f",{state}"
        else:
            df.loc[len(df.index)] = df_new.loc[i_row,:]

# df = df.drop_duplicates(subset='Name',keep=False)
df

In [99]:
df.to_csv(f'events-with-states', sep=',', index=False, encoding='utf-8')

In [5]:
states_convert = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}



In [46]:
query_list = {"Name": [], "Year": [], "State": []}
# df['Begin Date'] = pd.to_dateonly(df['Begin Date'])
# df['End Date'] = pd.to_datetime(df['End Date'])

for i_row in range(len(df.index)):
    states = df.loc[i_row,"State"].split(',')
    year = df.loc[i_row,'End Date']//10000
    if len(states) <= 3 and ((year > 2009 and year < 2019) or year == 2021):
        for state in states:
            query_list["Name"].append(df.loc[i_row,'Name']+' before')
            query_list["Year"].append(year)
            query_list["State"].append(states_convert[state])

            query_list["Name"].append(df.loc[i_row,'Name']+ ' after')
            query_list["Year"].append(year+1)
            query_list["State"].append(states_convert[state])


In [ ]:
# # reader = PyPDF2.PdfReader('us-climate-disasters.pdf')
# # cols = ["Name","Disaster","Begin Date","End Date","Total CPI-Adjusted Cost (Millions of Dollars)","Deaths","State"]
# df = pd.DataFrame(columns = cols)

# for file in file_list:
#     df_new = pd.read_csv(file,skiprows=1)
#     state = file.split('-')[3]
#     df_new['State'] = [state]*len(df_new.index)
#     df = pd.concat([df, df_new], axis=0)

# # df = df.drop_duplicates(subset='Name',keep=False)
# df
# df.to_csv(f'events-by-state-with-dup.csv', sep=',', index=False, encoding='utf-8')

In [47]:
df_state_fips = pd.read_json('state-fips.json')
df_state_fips.columns=df_state_fips.iloc[0]
df_state_fips.drop([0], inplace = True)

In [48]:
cols_2017 = ['Disaster Name','Year','State','S1903_C03_001E','S1903_C03_002E','S1903_C03_003E','S1903_C03_004E','S1903_C03_005E',\
           'S1903_C03_006E','S1903_C03_007E','S1903_C03_008E','S1903_C03_009E','S1903_C03_010E',\
           'S1903_C03_011E','S1903_C03_012E','S1903_C03_013E','S1903_C03_014E']

cols_2016 = ['Disaster Name','Year','State','S1903_C02_001E','S1903_C02_002E','S1903_C02_003E','S1903_C02_004E','S1903_C02_005E',\
           'S1903_C02_006E','S1903_C02_007E','S1903_C02_008E','S1903_C02_009E','S1903_C02_010E',\
           'S1903_C02_012E','S1903_C02_013E','S1903_C02_014E','S1903_C02_015E']

In [ ]:
####################################
##
##  Step 1: Connect to the server
##          Send a query
##          Collect and clean the 
##          results
####################################

####################################################
##In the following loop, we will query thenewsapi servers
##for all the topic names in the list
## We will then build a large csv file 
## where each article is a row
##
## From there, we will convert this data
## into a labeled dataframe
## so we can train and then test our DT
## model
####################################################

####################################################
## Build the URL and GET the results
## NOTE: At the bottom of this code
## commented out, you will find a second
## method for doing the following. This is FYI.
####################################################

## This is the endpoint - the server and 
## location on the server where your data 
## will be retrieved from

## TEST FIRST!
## We are about to build this URL:
## https://newsapi.org/v2/everything?apiKey=8f4134f7d0de43b8b49f91e22100f22b&q=bitcoin


endpoint_1="https://api.census.gov/data/"
endpoint_2="/acs/acs1/subject"
api_key='448115dcad7090fc4be722bf0f1406d1c232d4fd'
get_value = "NAME,group(S1903)"

df_census_2016 = pd.DataFrame(columns = cols_2016)
df_census_2017 = pd.DataFrame(columns = cols_2017)
# print(df_census.columns)

for i in range(len(query_list['Name'])):
# for i in range(10):
        
    year = query_list['Year'][i]
    print(i)
    row = df_state_fips[df_state_fips['NAME']==query_list['State'][i]]
    state_code = row.loc[row.index[0],'state']
   
    ## Dictionary Structure
    URLPost = {'get':get_value,
                'key':api_key,
                'for':f'state:{state_code}'
    }


    payload_str = "&".join("%s=%s" % (k,v) for k,v in URLPost.items())
    endpoint = endpoint_1+str(year)+endpoint_2
    # 'format=json&key=site:dummy+type:example+group:wheel'
    response = requests.get(endpoint, params=payload_str)

    # response=requests.get(endpoint, URLPost)
    # print(response.request.url)
    print(response)
    jsontxt = response.json()
    # print(jsontxt)
    if response.status_code == 200:

    ##############################################`######
        df_census = df_census_2017
        if year < 2017:
            df_census = df_census_2016
        ## Open the file for append
        # MyFILE=open(filename, "a") ## "a" for append to add stuff
        df_census.loc[len(df_census.index)] = 0   
        df_census.loc[len(df_census.index)-1,'Disaster Name'] = query_list['Name'][i]     
        df_census.loc[len(df_census.index)-1,'Year'] = year
        df_census.loc[len(df_census.index)-1,'State'] = query_list['State'][i]
        for j,col in enumerate(jsontxt[0]):
            if col in df_census.columns:
                df_census.loc[len(df_census.index)-1,col] = jsontxt[1][j]


In [57]:
variable_key_2017 = {'S1903_C03_001E':"Income, All",
                'S1903_C03_002E':"Income, White",
                'S1903_C03_003E':"Income, Black",
                'S1903_C03_004E':"Income, Native",
                'S1903_C03_005E':"Income, Asian",
                'S1903_C03_006E':"Income, Pacific Islander",
                'S1903_C03_007E':"Income, Other",
                'S1903_C03_008E':"Income, Two",
                'S1903_C03_009E':"Income, Hispanic",
                'S1903_C03_010E':"Income, White Alone",
                'S1903_C03_011E':"Income, 15 to 24",
                'S1903_C03_012E':"Income, 25 to 44",
                'S1903_C03_013E':"Income, 45 to 64",
                'S1903_C03_014E':"Income, 65 over"}
variable_key_2016 = {'S1903_C02_001E':"Income, All",
                'S1903_C02_002E':"Income, White",
                'S1903_C02_003E':"Income, Black",
                'S1903_C02_004E':"Income, Native",
                'S1903_C02_005E':"Income, Asian",
                'S1903_C02_006E':"Income, Pacific Islander",
                'S1903_C02_007E':"Income, Other",
                'S1903_C02_008E':"Income, Two",
                'S1903_C02_009E':"Income, Hispanic",
                'S1903_C02_010E':"Income, White Alone",
                'S1903_C02_012E':"Income, 15 to 24",
                'S1903_C02_013E':"Income, 25 to 44",
                'S1903_C02_014E':"Income, 45 to 64",
                'S1903_C02_015E':"Income, 65 over"}

df_census_2016_renamed = df_census_2016.rename(columns=variable_key_2016)
df_census_2017_renamed = df_census_2017.rename(columns=variable_key_2017)
df_census = pd.concat([df_census_2016_renamed, df_census_2017_renamed], axis=0)

In [63]:
df_census.to_csv(f'income-disaster.csv', sep=',', index=False, encoding='utf-8')

In [ ]:
# Replace negative values with "Income, All" values

# all = df_census['Income, All']
df_census.iloc[:,3:] = df_census.iloc[:,3:].astype('float')
df_census.iloc[:,3:] = df_census.iloc[:,3:].mask(df_census.iloc[:,3:] < 0)
df_census = df_census.fillna(df_census['Income, All'])
df_census


In [225]:
# Pivot by state occurrence for creating visualization
df_events_state = pd.read_csv("events-by-state-with-dup.csv")
df_events_state

df_state_num = pd.pivot_table(df_events_state, values='Name', index='State', aggfunc='count')
df_state_num = df_state_num.reset_index()
df_state_num.to_csv(f'state_num.csv', sep=',', index=False, encoding='utf-8')